In [117]:
import numpy as np
import cv2
from PIL import Image
from typing import Tuple, Optional, List, cast
from logging import getLogger

logger = getLogger(__name__)

INPUT_WIDTH: int = 640
INPUT_HEIGHT: int = 640

def _reorder_cv_image(src_image: np.ndarray) -> np.ndarray:
    assert src_image.ndim == 3
    assert src_image.shape[2] == 3 or src_image.shape[2] == 4
    dst_image: np.ndarray = src_image.copy()
    if dst_image.shape[2] == 3:
        dst_image = cv2.cvtColor(dst_image, cv2.COLOR_BGR2RGB)
    else:
        dst_image = cv2.cvtColor(dst_image, cv2.COLOR_BGRA2RGB)
    return dst_image

def _resize_input_image(
    image: np.ndarray, pad_value: Tuple[int, int, int] = (114, 114, 114)) -> np.ndarray:

    h, w, c = image.shape
    ratio_width: float = float(INPUT_WIDTH) / float(w)
    ratio_height: float = float(INPUT_HEIGHT) / float(h)

    ratio: float = 1.0
    dst_width: int = INPUT_WIDTH
    dst_height: int = INPUT_HEIGHT
    pad_width: int = 0
    pad_height: int = 0
    if ratio_width < ratio_height:
        # Letter box
        ratio = ratio_width
        dst_width = INPUT_WIDTH
        dst_height = min(round(h * ratio), INPUT_HEIGHT)
        pad_height = INPUT_HEIGHT - dst_height
    else:
        # Pillar box
        ratio = ratio_height
        dst_width = min(round(w * ratio), INPUT_WIDTH)
        dst_height = INPUT_HEIGHT
        pad_width = INPUT_WIDTH - dst_width
    pad_top: int = pad_height // 2
    pad_bottom: int = pad_height - pad_top
    pad_left: int = pad_width // 2
    pad_right: int = pad_width - pad_left

    logger.debug('{} {} {} {} {} {}'.format(
        dst_width, dst_height, pad_top, pad_bottom, pad_left, pad_right
    ))
    
    # Resize
    rescaled_image: np.ndarray = cv2.resize(
        src=image, dsize=(dst_width, dst_height), interpolation=cv2.INTER_LINEAR)

    # Pad
    padded_image: np.ndarray = cv2.copyMakeBorder(src=rescaled_image, 
        top=pad_top, bottom=pad_bottom, left=pad_left, right=pad_right, 
        borderType=cv2.BORDER_CONSTANT, value=pad_value)

    # Debug
    # cv2.imwrite('padded_image.png', padded_image)

    return padded_image

def _preprocess_images(images: List[np.ndarray]) -> np.ndarray:
    cv_images: List[np.ndarray] = [_resize_input_image(img) for img in images]
    np_images: List[np.ndarray] = [
        np.ascontiguousarray(img.transpose(2, 0, 1), dtype='uint8') for img in cv_images
    ]
    return np.array(np_images)

def preprocess_cv_images(images: List[np.ndarray]) -> np.ndarray:
    cv_images: List[np.ndarray] = [_reorder_cv_image(img) for img in images]
    return _preprocess_images(cv_images).astype(np.float32)

In [111]:
import numpy as np
from attrdict import AttrDict

import tritonclient.http as httpclient
from tritonclient.utils import InferenceServerException
import tritonclient.grpc.model_config_pb2 as mc

class TritonClientError(Exception):
    pass


class TritonClient():

    def __init__(self, url='localhost:8000'):
        self.request = None
        self.response = None

        try:
            self.client = httpclient.InferenceServerClient(
                url=url, verbose=False, concurrency=1
            )
        except Exception as e:
            print('could not create client: {}'.format(e))
            raise TritonClientError(str(e))

    def infer(self, image, class_count=0):
        # if self.max_batch_size > 0:
        #     image = image[np.newaxis, :]

        inputs = [httpclient.InferInput("image", (1,3,640,640), "FP32"), httpclient.InferInput("scale_factor", (1,2), "FP32")]
        inputs[0].set_data_from_numpy(image)
        inputs[1].set_data_from_numpy(np.array([[1.0, 1.0]]).astype(np.float32))

        outputs = [httpclient.InferRequestedOutput("multiclass_nms3_0.tmp_0", class_count=class_count)]

        try:
            self.request = self.client.async_infer(
                "ppyoloe_detection", inputs, request_id='0',
                model_version="1", outputs=outputs
            )
        except InferenceServerException as e:
            print('Inference failed: {}'.format(e))
            raise TritonClientError(str(e))

    def get_results(self):
        if self.request is None:
            return None

        self.response = self.request.get_result()
        output_array = self.response.as_numpy("multiclass_nms3_0.tmp_0")
        # if self.max_batch_size <= 0:
        #     output_array = output_array[np.newaxis, :]

        return output_array

In [130]:
im = cv2.imread("4.png")
# blob = cv2.dnn.blobFromImage(im, 1 / 255.0, (640, 640), swapRB=True, crop=False)
im = preprocess_cv_images([im])
tr = TritonClient()
tr.infer(im)
res = tr.get_results()
# tr.parse_model("ppyoloe_detection")


In [131]:
print(res.shape)
print(res)

(100, 6)
[[0.00000000e+00 9.71310198e-01 1.32107330e+02 2.08705109e+02
  3.94052155e+02 5.11019867e+02]
 [0.00000000e+00 9.38606083e-01 3.91318573e+02 3.01075958e+02
  4.78144012e+02 3.76876801e+02]
 [0.00000000e+00 9.33101654e-01 4.80625427e+02 3.23471893e+02
  5.54031982e+02 3.85737732e+02]
 [0.00000000e+00 9.24767315e-01 3.84637573e+02 3.21384308e+02
  4.33129395e+02 3.85032288e+02]
 [0.00000000e+00 9.19485748e-01 9.28017883e+01 3.05717804e+02
  1.56467590e+02 3.78782410e+02]
 [0.00000000e+00 8.88354421e-01 5.75124268e+02 2.64408081e+02
  6.20672302e+02 2.96625519e+02]
 [0.00000000e+00 8.82210970e-01 5.33203003e+02 1.94389130e+02
  5.61036438e+02 2.20523788e+02]
 [0.00000000e+00 8.67794096e-01 5.10744110e+02 2.22244400e+02
  5.46188232e+02 2.54436340e+02]
 [0.00000000e+00 8.65806222e-01 4.88265076e+02 2.73949951e+02
  5.27965576e+02 3.08006622e+02]
 [0.00000000e+00 8.62305522e-01 4.98014923e+02 2.38375671e+02
  5.37854370e+02 2.70311340e+02]
 [0.00000000e+00 8.61234307e-01 5.6900543

In [132]:
im = cv2.imread("4.png")
height, width, _ = im.shape
height_border = width - height if width >= height else 0
width_border = height - width if height >= width else 0
print(height, width)
color = (255,0,0)
thickness = 1
# print(res)
for rec in res:
    if rec[1] > 0.5:
        start_point = (int(rec[2] / 640.0 * (width + width_border) - width_border // 2), int(rec[3] / 640.0 * (height + height_border) - height_border // 2)) 
        end_point = (int(rec[4] / 640.0 * (width + width_border) - width_border // 2), int(rec[5] / 640.0 * (height + height_border) - height_border // 2))
        print(start_point, end_point)
        im = cv2.rectangle(im, start_point, end_point, color, thickness)  
cv2.imwrite("test.png", im)

805 1205
(248, 192) (741, 762)
(736, 366) (900, 509)
(904, 409) (1043, 526)
(724, 405) (815, 524)
(174, 375) (294, 513)
(1082, 297) (1168, 358)
(1003, 165) (1056, 215)
(961, 218) (1028, 279)
(919, 315) (994, 379)
(937, 248) (1012, 308)
(1071, 250) (1159, 328)
(855, 373) (937, 453)
(973, 124) (1034, 170)
(745, 244) (842, 338)
(938, 293) (1007, 353)
(870, 344) (964, 421)
(1104, 184) (1163, 230)
(1100, 156) (1163, 213)
(784, 336) (865, 379)
(1119, 201) (1188, 261)
(1092, 227) (1160, 272)
(801, 232) (868, 301)
(1015, 361) (1144, 447)
(949, 359) (1205, 799)
(1060, 106) (1112, 155)
(1078, 120) (1134, 177)
(718, 338) (771, 389)
(966, 86) (1020, 129)
(1125, 246) (1181, 305)


True

In [ ]:
# Before my work
# After from YOLO site

In [17]:
from ultralytics import YOLO
# Load a model
model = YOLO('yolov8n.pt')  # load an official model

# Export the model
onnx_file = model.export(format='onnx', dynamic=True)

Ultralytics YOLOv8.0.229 🚀 Python-3.8.18 torch-2.1.2 CPU (AMD Ryzen 7 7700X 8-Core Processor)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 0.3s, saved as 'yolov8n.onnx' (12.1 MB)

Export complete (1.5s)
Results saved to /home/user/ml
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


In [18]:
from pathlib import Path

# Define paths
triton_repo_path = Path('tmp') / 'triton_repo'
triton_model_path = triton_repo_path / 'yolo'

# Create directories
(triton_model_path / '1').mkdir(parents=True, exist_ok=True)

In [19]:
from pathlib import Path

# Move ONNX model to Triton Model path
Path(onnx_file).rename(triton_model_path / '1' / 'model.onnx')

# Create config file
(triton_model_path / 'config.pbtxt').touch()

In [23]:
import subprocess
import time
import contextlib

from tritonclient.http import InferenceServerClient

# Define image https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver
# tag = 'nvcr.io/nvidia/tritonserver:21.10-py3'  # 6.4 GB

# Pull the image
# subprocess.call(f'docker pull {tag}', shell=True)

# Run the Triton server and capture the container ID
container_id = subprocess.check_output(
    f'docker run -d --runtime=nvidia --gpus all --rm -v /home/user/ml/tmp/triton_repo:/models -p 8000:8000 nvcr.io/nvidia/tritonserver:22.12-py3 tritonserver --model-repository=/models',
    shell=True).decode('utf-8').strip()

# Wait for the Triton server to start
triton_client = InferenceServerClient(url='localhost:8000', verbose=False, ssl=False)

# Wait until model is ready
for _ in range(10):
    with contextlib.suppress(Exception):
        assert triton_client.is_model_ready(model_name)
        break
    time.sleep(1)

In [33]:
from ultralytics import YOLO
import numpy as np

# Load the Triton Server model
model = YOLO(f'http://localhost:8000/yolo', task='detect')

# Run inference on the server
results = model('4.png')
# print(results)



image 1/1 /home/user/ml/4.png: 640x640 8 class0s, 19 class2s, 1 class5, 1 class7, 16.0ms
Speed: 1.3ms preprocess, 16.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


In [34]:
subprocess.call(f'docker kill {container_id}', shell=True)

62dd24dd03a8


0